In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="Spark1")

In [2]:
# read in genre and artist
source_rdd = sc.textFile("artists_with_genres.csv",100)

In [3]:
# see what we got
source_rdd.take(5)

[u',artist,name,genre',
 u'0,03098741-08b3-4dd7-b3f6-1b0bfa2c879c,Liars,alternative rock',
 u'1,69c4cc43-8163-41c5-ac81-30946d27bb69,CunninLynguists,american',
 u'2,7a2e6b55-f149-4e74-be6a-30a1b1a387bb,The Desert Sessions,rock and indie',
 u'3,7002bf88-1269-4965-a772-4ba1e7a91eaa,Glenn Gould,canadian pianist']

In [4]:
# remove header
header = source_rdd.first()
source_no_head_rdd = source_rdd.filter(lambda x: x != header)

In [5]:
# create artists_rdd with key = artist_id, value = [genre]
artists_rdd = source_no_head_rdd.map(lambda x: x.split(','))\
                                .map(lambda (row_id,artist_id,name,genre): (artist_id,[genre]))

In [6]:
artists_rdd.take(5)

[(u'03098741-08b3-4dd7-b3f6-1b0bfa2c879c', [u'alternative rock']),
 (u'69c4cc43-8163-41c5-ac81-30946d27bb69', [u'american']),
 (u'7a2e6b55-f149-4e74-be6a-30a1b1a387bb', [u'rock and indie']),
 (u'7002bf88-1269-4965-a772-4ba1e7a91eaa', [u'canadian pianist']),
 (u'dbf7c761-e332-467b-b4d9-aafe06bbcf8f', [u'rock and indie'])]

In [7]:
# create rdd where key = user, value = list of (genre, avg # of plays per artist of that genre)
# read in train
source_train_rdd = sc.textFile("train.csv",100)

In [8]:
source_train_rdd.take(5)

[u'user,artist,plays',
 u'eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,5a8e07d5-d932-4484-a7f7-e700793a9c94,554',
 u'44ce793a6cd9d20f13f4a576a818ef983314bb5d,a3a92047-be1c-4f3e-8960-c4f8570984df,81',
 u'da9cf3f557161d54b76f24db64be9cc76db008e3,eeb1195b-f213-4ce1-b28c-8565211f8e43,708',
 u'8fa49ab25d425edcf05d44bfc1d5aea895287d81,a1419808-65d3-4d40-998c-1a0bac65eabc,265']

In [9]:
# remove header
train_header = source_train_rdd.first()
source_train_no_head_rdd = source_train_rdd.filter(lambda x: x != train_header)

In [10]:
source_train_no_head_rdd.take(5)

[u'eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,5a8e07d5-d932-4484-a7f7-e700793a9c94,554',
 u'44ce793a6cd9d20f13f4a576a818ef983314bb5d,a3a92047-be1c-4f3e-8960-c4f8570984df,81',
 u'da9cf3f557161d54b76f24db64be9cc76db008e3,eeb1195b-f213-4ce1-b28c-8565211f8e43,708',
 u'8fa49ab25d425edcf05d44bfc1d5aea895287d81,a1419808-65d3-4d40-998c-1a0bac65eabc,265',
 u'b85fcaef67d2669cd99b334b5e8c8705263db2cf,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,220']

In [11]:
train_rdd = source_train_no_head_rdd.map(lambda x: x.split(','))\
                                    .map(lambda (user_id,artist_id,plays):(artist_id,[user_id,plays]))


In [12]:
train_rdd.take(5)

[(u'5a8e07d5-d932-4484-a7f7-e700793a9c94',
  [u'eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03', u'554']),
 (u'a3a92047-be1c-4f3e-8960-c4f8570984df',
  [u'44ce793a6cd9d20f13f4a576a818ef983314bb5d', u'81']),
 (u'eeb1195b-f213-4ce1-b28c-8565211f8e43',
  [u'da9cf3f557161d54b76f24db64be9cc76db008e3', u'708']),
 (u'a1419808-65d3-4d40-998c-1a0bac65eabc',
  [u'8fa49ab25d425edcf05d44bfc1d5aea895287d81', u'265']),
 (u'a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432',
  [u'b85fcaef67d2669cd99b334b5e8c8705263db2cf', u'220'])]

In [13]:
# concatenate the artists_rdd and train_with_genres_rdd
# then create new rdd with key = artist, value = ([user_id,plays],[genre])
train_with_genres_rdd = train_rdd.join(artists_rdd)

In [14]:
train_with_genres_rdd.take(5)

[(u'adc0f033-95c2-4e0b-87bc-c23ed3f26ce6',
  ([u'ff667736eee24f2e0a6128330fda4bfeb162d871', u'393'], [u'hard rock'])),
 (u'adc0f033-95c2-4e0b-87bc-c23ed3f26ce6',
  ([u'1eb59a03fa35670a5462fd79da7a7e32190aae6a', u'663'], [u'hard rock'])),
 (u'adc0f033-95c2-4e0b-87bc-c23ed3f26ce6',
  ([u'c15b0b67ede275c81e017139cec8dd62d3e8c203', u'53'], [u'hard rock'])),
 (u'adc0f033-95c2-4e0b-87bc-c23ed3f26ce6',
  ([u'3a1ba3598566883c8e9a82ac6b49609576148d2f', u'571'], [u'hard rock'])),
 (u'adc0f033-95c2-4e0b-87bc-c23ed3f26ce6',
  ([u'aadd96164221be83f26d30210468be635e5c4d2f', u'311'], [u'hard rock']))]

In [15]:
# now create rdd with key = (user_id,genre), value = (plays,1)
# then reduce by key to get key = (user_id, genre), value = (total_plays, count)
# then map to key = (user,genre), value = (average number of plays)
user_genre_to_plays = train_with_genres_rdd.map(lambda (artist_id,x): ((x[0][0],x[1][0]),(int(x[0][1]),1)))\
                                            .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
                                            .map(lambda x: (x[0],x[1][0]/float(x[1][1])))
user_genre_to_plays.take(5)

[((u'53224e671b8b43b2cf7d32b23fa788fe8ca073d8', u'brazilian'), 349.0),
 ((u'4e42cdf2688f448f0cf8c26b113b5cb0c5a40534', u'indie'), 27.0),
 ((u'57cbc256478ea9ff3615de9966bb46d029a36c31', u'alternative pop'), 228.0),
 ((u'2b13fa41b81093e153c71e1f06a3bcea47be4d6c', u'punk rock'), 3057.0),
 ((u'2735cc1185e3c55e54c84425d6cbf52baa39c949', u''), 7.0)]

In [17]:
# create strings to write to file
string_rdd = user_genre_to_plays.map(lambda x: ','.join([x[0][0],x[0][1],str(x[1])]))
string_rdd.saveAsTextFile('user_genre_avg_plays.csv')

In [18]:
# merge all partitions csvs into one csv
filenames = ['part-0000'+str(i) for i in xrange(10)]
filenames2 = ['part-000'+str(i) for i in xrange(10,100)]
filenames3 = ['part-00'+str(i) for i in xrange(100,200)]
filenames.extend(filenames2)
filenames.extend(filenames3)
combined_out_fp = open('train_user_genre_avg_plays.csv','w')
for filename in filenames:
    in_fp = open('./user_genre_avg_plays.csv/'+filename,'r')
    for line in in_fp:
        if line not in ['','\n']:
            combined_out_fp.write(line)
    in_fp.close()
combined_out_fp.close()